In [433]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import pandas as pd
import pyreadr
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
import warnings
warnings.filterwarnings("ignore")

In [434]:
result = pyreadr.read_r('gene_exp.rds')
gene_exp = result[None]
result = pyreadr.read_r('metadata_patient.rds') 
meta = result[None]
result = pyreadr.read_r('gc_columbia.rds') 
gc_columbia = result[None]
result = pyreadr.read_r('gc_broad.rds')
gc_broad = result[None]
result = pyreadr.read_r('gc_uw.rds')
gc_uw = result[None]
result = pyreadr.read_r('tpm_negative_mean.rds') 
tpm_negative_mean = result[None]
result = pyreadr.read_r('tpm_negative_sd.rds') 
tpm_negative_sd = result[None]
result = pyreadr.read_r('gene_combined.rds')
gene_combined = result[None]


In [435]:
negative_samples = meta[meta['diagnosis'] == 'Negative']
negative_sample_ids = negative_samples['sample.id'].values
gene_exp_filtered = gene_exp.drop(columns=negative_sample_ids)

In [436]:
gene_exp = (gene_exp_filtered - np.array(tpm_negative_mean))/np.array(tpm_negative_sd)
common_rows = gene_exp.index.intersection(gene_combined.iloc[:, 0])
gene_exp_filtered = gene_exp.loc[common_rows]
merged = meta.merge(gene_exp_filtered.T, left_on='sample.id', right_index=True)
def average_range(s):
    if isinstance(s, str) and '-' in s:
        split_s = s.split('-')
        return (float(split_s[0]) + float(split_s[1])) / 2
    else:
        return float(s)

merged.iloc[:, 2] = merged.iloc[:, 2].apply(average_range)
merged.iloc[:, 2] = merged.iloc[:, 2].fillna(merged.iloc[:, 2].mean())

In [425]:
selected_columns = merged.iloc[:, 12:]
abs_values = selected_columns.abs()
average_abs_values = abs_values.mean()
ranked_average_abs_values = average_abs_values.sort_values(ascending=False)

In [370]:
ranked_average_abs_values[0:]

ADM       1.153716
C1QC      1.134224
MAFF      1.098387
C1QA      1.068627
UGT2A2    1.066048
            ...   
HSFX3     0.371778
PMCH      0.364839
FOXF2     0.322847
GREM1     0.295139
NAT8L     0.258984
Length: 18035, dtype: float64

In [339]:
best_model = [None]*20
accuracy = [None]*20
auroc = [None]*20

for i in range(20):
    X = merged[ranked_average_abs_values.index[0:1 + i]].values
    y = merged['diagnosis'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    print(X.shape)
    params = {'hidden_layer_sizes': [(10,), (15,), (20,)],
              'alpha': [1e-4, 1e-5, 1e-6],
              'solver': ['sgd', 'adam'],
              'random_state': [123]}
    model = MLPClassifier()

    grid_search = GridSearchCV(model, param_grid=params, cv=5)
    grid_search.fit(X_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Best accuracy: ", grid_search.best_score_)

    best_model[i] = grid_search.best_estimator_
    pred = best_model[i].predict(X_test)
    accuracy[i] = accuracy_score(y_test, pred)
    print("Test accuracy: ", accuracy[i])
    pred_probs = best_model[i].predict_proba(X_test)
    # Calculate the AUROC score, expected over 70%
    auroc[i] = roc_auc_score(y_test, pred_probs[:, 1])
    print("AUROC: ", auroc[i])


(247, 1)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.6903846153846155
Test accuracy:  0.76
AUROC:  0.4451754385964912
(247, 2)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.6903846153846155
Test accuracy:  0.76
AUROC:  0.5241228070175439
(247, 3)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'adam'}
Best accuracy:  0.7156410256410257
Test accuracy:  0.76
AUROC:  0.7149122807017544
(247, 4)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.6905128205128206
Test accuracy:  0.76
AUROC:  0.6535087719298246
(247, 5)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.7206410256410256
Test accuracy:  0.64
AUROC:  0.6929824561403508
(247, 6)
Best parameters

In [379]:
# Age included as the first feature

_best_model = [None]*21
_accuracy = [None]*21
_auroc = [None]*21

for i in range(21):
    selected_columns = merged[ranked_average_abs_values.index[0: i]]
    selected_columns.insert(0, 'age', merged['age'])
    X = selected_columns.values
    y = merged['diagnosis'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    print(X.shape)
    params = {'hidden_layer_sizes': [(10,), (15,), (20,)],
              'alpha': [1e-4, 1e-5, 1e-6],
              'solver': ['sgd', 'adam'],
              'random_state': [123]}
    model = MLPClassifier()

    grid_search = GridSearchCV(model, param_grid=params, cv=5)
    grid_search.fit(X_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Best accuracy: ", grid_search.best_score_)

    _best_model[i] = grid_search.best_estimator_
    pred = _best_model[i].predict(X_test)
    _accuracy[i] = accuracy_score(y_test, pred)
    print("Test accuracy: ", _accuracy[i])
    pred_probs = _best_model[i].predict_proba(X_test)
    # Calculate the AUROC score, expected over 70%
    _auroc[i] = roc_auc_score(y_test, pred_probs[:, 1])
    print("AUROC: ", _auroc[i])


(247, 1)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'adam'}
Best accuracy:  0.7203846153846154
Test accuracy:  0.64
AUROC:  0.5592105263157895
(247, 2)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'random_state': 123, 'solver': 'adam'}
Best accuracy:  0.6903846153846155
Test accuracy:  0.76
AUROC:  0.5657894736842106
(247, 3)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'adam'}
Best accuracy:  0.6903846153846155
Test accuracy:  0.76
AUROC:  0.5021929824561403
(247, 4)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.6951282051282052
Test accuracy:  0.7
AUROC:  0.7236842105263158
(247, 5)
Best parameters:  {'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'random_state': 123, 'solver': 'sgd'}
Best accuracy:  0.7002564102564103
Test accuracy:  0.68
AUROC:  0.6644736842105263
(247, 6)
Best parameter

In [442]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

X = merged.iloc[:,[2]].values
y = merged.iloc[:, 10].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train) 
y_predict = classifier.predict(X_test)
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))
pred_probs = classifier.predict_proba(X_test)
auroc = roc_auc_score(y_test, pred_probs[:, 1])
print("AUROC: ", auroc)

# params = {'hidden_layer_sizes': [(10,), (15,), (20,)],
#               'alpha': [1e-4, 1e-5, 1e-6],
#               'solver': ['sgd', 'adam'],
#               'random_state': [123]}
# model = MLPClassifier()
# grid_search = GridSearchCV(model, param_grid=params, cv=5)
# grid_search.fit(X_train, y_train)
# print("Best parameters: ", grid_search.best_params_)
# print("Best accuracy: ", grid_search.best_score_)
# best_model = grid_search.best_estimator_
# pred = best_model.predict(X_test)
# accuracy = accuracy_score(y_test, pred)
# print("Test accuracy: ", accuracy)
# pred_probs = best_model.predict_proba(X_test)
# auroc = roc_auc_score(y_test, pred_probs[:, 1])
# print("AUROC: ", auroc)



[[30  8]
 [ 8  4]]
              precision    recall  f1-score   support

        Mild       0.79      0.79      0.79        38
      Severe       0.33      0.33      0.33        12

    accuracy                           0.68        50
   macro avg       0.56      0.56      0.56        50
weighted avg       0.68      0.68      0.68        50

AUROC:  0.569078947368421


In [471]:
params = {
    'hidden_layer_sizes': [(20,), (15, 10,), (10, 10,), (20, 10, 5), (30, 20, 10)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [1e-4, 1e-5, 1e-6],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'random_state': [123],
}
model = MLPClassifier()
grid_search = GridSearchCV(model, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)
print("Best parameters: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)
best_model = grid_search.best_estimator_
pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print("Test accuracy: ", accuracy)
pred_probs = best_model.predict_proba(X_test)
auroc = roc_auc_score(y_test, pred_probs[:, 1])
print(best_model)
print("AUROC: ", auroc)
num_hidden_layers = len(best_model.hidden_layer_sizes)
print("Number of hidden layers: ", num_hidden_layers)
print("Number of neurons in each hidden layer: ", best_model.hidden_layer_sizes)



Best parameters:  {'activation': 'logistic', 'alpha': 1e-06, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'random_state': 123, 'solver': 'lbfgs'}
Best accuracy:  0.7715384615384615
Test accuracy:  0.66
MLPClassifier(activation='logistic', alpha=1e-06, hidden_layer_sizes=(10, 10),
              random_state=123, solver='lbfgs')
AUROC:  0.5745614035087719
Number of hidden layers:  1
Number of neurons in each hidden layer:  (10, 10)


In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

fpr, tpr, thresholds = roc_curve(y_test, pred_probs[:, 1], pos_label='Severe')

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auroc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
